In [71]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import cv2
import numpy as np

In [72]:
batch_size = 16

In [79]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '/Users/danielkashkett/Desktop/EyeProject/data/micronet/',
  validation_split=0.2,
    color_mode="grayscale",
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '/Users/danielkashkett/Desktop/EyeProject/data/micronet/',
  validation_split=0.2,
  color_mode="grayscale",
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

class_names = train_ds.class_names
print(class_names)

Found 17996 files belonging to 5 classes.
Using 14397 files for training.
Found 17996 files belonging to 5 classes.
Using 3599 files for validation.
['center', 'down', 'left', 'right', 'up']


In [80]:
num_classes = 5
img_height = 30
img_width = 30
input_layer = keras.Input(shape=(img_height, img_width, 1))
x = keras.layers.experimental.preprocessing.Rescaling(1./255)(input_layer)
# x = keras.layers.Lambda(lambda x: tf.image.random_brightness(x, max_delta=0.2))(x)
# x = keras.layers.Lambda(lambda x: tf.image.random_contrast(x, lower=0.1, upper=0.9))(x)
x = keras.layers.Conv2D(32, 1, padding='same', activation='relu')(x)
x = keras.layers.MaxPooling2D()(x)
x = keras.layers.Conv2D(64, 1, padding='same', activation='relu')(x)
x = keras.layers.MaxPooling2D()(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Conv2D(128, 1, padding='same', activation='relu')(x)
x = keras.layers.MaxPooling2D()(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(512, activation='relu')(x)
output_layer = keras.layers.Dense(num_classes, activation='softmax')(x)

model = keras.Model(inputs=input_layer, outputs=output_layer, name="jupiter3")

opt = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,name='Adam')
# opt = tf.keras.optimizers.SGD(lr=0.0001, momentum=0.9)
# model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    
model.compile(optimizer=opt, 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
   
model.summary()

Model: "jupiter3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_29 (InputLayer)        [(None, 30, 30, 1)]       0         
_________________________________________________________________
rescaling_28 (Rescaling)     (None, 30, 30, 1)         0         
_________________________________________________________________
conv2d_67 (Conv2D)           (None, 30, 30, 32)        64        
_________________________________________________________________
max_pooling2d_67 (MaxPooling (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_68 (Conv2D)           (None, 15, 15, 64)        2112      
_________________________________________________________________
max_pooling2d_68 (MaxPooling (None, 7, 7, 64)          0         
_________________________________________________________________
dropout_37 (Dropout)         (None, 7, 7, 64)          0  

In [81]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=7, 
  batch_size =batch_size
)

Epoch 1/7
900/900 [==============================] - 16s 18ms/step - loss: 1.3412 - accuracy: 0.5600 - val_loss: 1.2106 - val_accuracy: 0.6899
Epoch 2/7
900/900 [==============================] - 11s 13ms/step - loss: 1.0996 - accuracy: 0.8094 - val_loss: 1.0061 - val_accuracy: 0.9014
Epoch 3/7
900/900 [==============================] - 9s 10ms/step - loss: 1.0218 - accuracy: 0.8854 - val_loss: 0.9821 - val_accuracy: 0.9236
Epoch 4/7
900/900 [==============================] - 9s 10ms/step - loss: 0.9940 - accuracy: 0.9131 - val_loss: 0.9610 - val_accuracy: 0.9461
Epoch 5/7
900/900 [==============================] - 9s 10ms/step - loss: 0.9799 - accuracy: 0.9257 - val_loss: 0.9542 - val_accuracy: 0.9547
Epoch 6/7
900/900 [==============================] - 9s 10ms/step - loss: 0.9680 - accuracy: 0.9380 - val_loss: 0.9549 - val_accuracy: 0.9492
Epoch 7/7
900/900 [==============================] - 9s 10ms/step - loss: 0.9653 - accuracy: 0.9406 - val_loss: 0.9435 - val_accuracy: 0.9644


In [82]:
model.save('./Models/micronet')

INFO:tensorflow:Assets written to: ./Models/micronet/assets


In [2]:
model = tf.keras.models.load_model('./Models/jupiter1')

In [55]:
img1 = cv2.imread('../data/eye_imgs/right/a1.jpg' ,cv2.IMREAD_UNCHANGED)
img2 = cv2.imread('../data/eye_imgs/left/a10.jpg' ,cv2.IMREAD_UNCHANGED)
# img = img.reshape((100,100,1))
# img = np.expand_dims(img,0)

In [56]:
predict_both(left = img1, right = img2)

3

In [24]:
def joint_prediction():
    pass

In [42]:
img1 = np.expand_dims(img1,0)
img2 = np.expand_dims(img2,0)

In [43]:
a = np.concatenate((img1,img2))

In [27]:
a.shape

(2, 100, 100)

In [45]:
model.predict(a)

array([ 2.0141094, -0.907617 ,  3.9984462, -2.9328032,  1.9947188],
      dtype=float32)

In [53]:
def predict_both(left, right):
    img1 = np.expand_dims(left,0)
    img2 = np.expand_dims(right,0)
    concat = np.concatenate((img1,img2))
    classes = model.predict_classes(concat)
    if classes[0] == classes[1]:
        return classes[0]
    else:
        pred = model.predict(concat)
        a = (pred[0].argmax(), pred[0].max())
        b = (pred[1].argmax(), pred[1].max())
        if a[1] > b[1]:
            return a[0]
        else:
            return b[0]

In [ ]:
def predict_frame(self, eye_img):
    img = eye_img.copy()
    batch = np.expand_dims(img,0)
    prediction = self.model.predict_classes(batch)[0]
    return prediction

In [211]:
class_names

['center', 'down', 'left', 'right', 'up']

In [212]:
model.predict_classes(img)[0]

2